In [73]:
import pandas as pd
from numpy import nan, mean
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# Home Data pipeline

See [readme](../README.md) 2do section for next steps

In [74]:
# load data

global training___data, validation_data, training___price, validation_price, full_data, full_price, test_data, original_data, original_test_data
def load_data():
    global training___data, validation_data, training___price, validation_price, full_data, full_price, test_data, original_data, original_test_data
    test_data = pd.read_csv("./test.csv")
    original_test_data = test_data.copy()
    original_data = pd.read_csv("./train.csv")
    full_price = original_data.SalePrice
    training___data, validation_data, training___price, validation_price = train_test_split(original_data, full_price)
    training___data = training___data.drop("SalePrice", axis=1)
    validation_data = validation_data.drop("SalePrice", axis=1)
    full_data = original_data.drop("SalePrice", axis=1)

load_data()
training___data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
778,779,90,RH,60.0,8400,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,SawyerW,Feedr,Norm,Duplex,1Story,5,5,1977,1977,Gable,CompShg,Plywood,Plywood,BrkFace,320.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0,NaN,0,0,0,GasA,TA,Y,SBrkr,2020,0,0,2020,0,0,2,0,4,2,TA,10,Typ,2,TA,Detchd,1977.0,Unf,2,630,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,10,2007,WD,Normal
273,274,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,6,6,1958,1988,Hip,CompShg,Wd Sdng,Wd Sdng,BrkCmn,183.0,TA,TA,CBlock,TA,TA,No,Rec,620,LwQ,620,0,1240,GasA,Gd,Y,SBrkr,1632,0,0,1632,1,0,2,0,3,1,TA,6,Min1,1,Gd,Attchd,1958.0,RFn,1,338,TA,TA,Y,289,0,0,0,0,0,NaN,MnPrv,NaN,0,4,2009,WD,Normal
929,930,60,RL,NaN,13006,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,1997,1997,Gable,CompShg,HdBoard,HdBoard,BrkFace,285.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,964,964,GasA,Gd,Y,SBrkr,993,1243,0,2236,0,0,2,1,4,1,Gd,8,Typ,1,TA,BuiltIn,1997.0,Fin,2,642,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,11,2006,WD,Normal
417,418,70,RL,86.0,22420,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Crawfor,Feedr,Norm,1Fam,2Story,6,6,1918,1950,Hip,CompShg,Wd Sdng,Stucco,NaN,0.0,TA,TA,BrkTil,Gd,TA,No,BLQ,1128,Unf,0,242,1370,GasW,TA,N,FuseA,1370,1254,0,2624,1,0,2,1,4,1,TA,10,Typ,1,Gd,Detchd,1918.0,Unf,3,864,TA,TA,N,0,0,0,0,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal
655,656,160,RM,21.0,1680,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,BrDale,Norm,Norm,Twnhs,2Story,6,5,1971,1971,Gable,CompShg,HdBoard,ImStucc,BrkFace,381.0,TA,TA,CBlock,TA,TA,No,Unf,0,Unf,0,525,525,GasA,TA,Y,SBrkr,525,567,0,1092,0,0,1,1,3,1,TA,6,Typ,0,NaN,Detchd,1971.0,Unf,1,264,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,Family
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1273,1274,80,RL,124.0,11512,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Edwards,Norm,Norm,1Fam,SLvl,6,7,1959,2006,Gable,CompShg,Plywood,Plywood,BrkFace,84.0,TA,TA,CBlock,TA,TA,Av,ALQ,719,Unf,0,300,1019,GasA,Gd,Y,SBrkr,1357,0,0,1357,1,0,1,0,2,1,Ex,5,Typ,1,Gd,Basment,1959.0,RFn,1,312,TA,TA,Y,0,0,0,0,163,0,NaN,GdPrv,NaN,0,5,2008,WD,Normal
481,482,20,RL,72.0,11846,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,1Story,9,5,2003,2004,Hip,CompShg,VinylSd,VinylSd,BrkFace,562.0,Gd,TA,PConc,Ex,TA,Gd,GLQ,1567,Unf,0,225,1792,GasA,Ex,Y,SBrkr,1792,0,0,1792,1,0,2,0,2,1,Ex,6,Typ,1,Gd,Attchd,2003.0,Fin,3,874,TA,TA,Y,206,49,0,0,0,0,NaN,NaN,NaN,0,8,2006,WD,Normal
268,269,30,RM,71.0,6900,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,6,1940,1955,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,ALQ,403,Rec,125,212,740,GasA,Ex,Y,SBrkr,778,0,0,778,0,0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1966.0,Fin,1,924,Ex,Ex,Y,0,25,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1052,1053,60,RL,100.0,9500,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Artery,Norm,1Fam,2Story,6,6,1964,1978,Gable,CompShg,VinylSd,VinylSd,BrkCmn,272.0,TA,TA,CBlock,TA,TA,No,Rec,442,Unf,0,374,816,GasA,TA,Y,SBrkr,1127,850,0,1977,0,1,1,1,4,1,TA,9,Typ,

In [75]:
full_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,NaN,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,GasA,Gd,Y,FuseA,1078,0,0,1078,1,0,1,0,2,1,Gd,5,Typ,0,NaN,Attchd,1950.0,Unf,1,2

In [76]:
# define possible column transformations

is_nan = lambda x: x != x
equals = lambda a, b: (is_nan(a) and is_nan(b)) or a == b
all_unique_values = lambda column_name: set(training___data[column_name].unique()) | set(validation_data[column_name].unique())

def drop(column_name, data_set, *_):
    data_set.drop(column_name, axis=1, inplace=True)
    return data_set

def drop_row(column_name, data_set, condition):
    return data_set[~data_set[column_name].apply(condition)]

def one_hot(column_name, data_set, options):
    keys = options if options else data_set[column_name].unique()
    for key in keys:
        data_set[f"{column_name}_{key}"] = data_set[column_name].apply(lambda x: 1 if equals(x, key) else 0)
    return drop(column_name, data_set)

def apply(column_name, data_set, function):
    data_set[column_name] = data_set[column_name].apply(function)
    return data_set

def re_map(column_name, data_set, mapping):
    data_set[column_name] = data_set[column_name].apply(lambda x: mapping[x])
    return data_set

def MSZoning(column_name, data_set, *_):
    options = dict(
        # A="Agriculture",
        C="Commercial",
        FV="Floating Village Residential",
        # I="Industrial",
        RH="Residential High Density",
        RL="Residential Low Density",
        # RP="Residential Low Density Park",
        RM="Residential Medium Density",
    )
    for key, name in options.items():
        if key == "C":
            continue
        data_set[f"zoned_{name}"] = data_set[column_name].apply(lambda x: 1 if x in [key, "C"] else 0)
    return drop(column_name, data_set)

def masonry(column_name, data_set, *_):
    options = dict(
        BrkCmn="Brick Common",
        BrkFace="Brick Face",
        CBlock="Cinder Block",
        Stone="Stone",
    )
    for key in options:
        data_set[f"masonry {options[key]}"] = 0
        for id in range(len(data_set)):
            if data_set[column_name].iloc[id] == key:
                data_set[f"masonry {options[key]}"].iat[id] = data_set["MasVnrArea"].iloc[id]
    
    data_set.drop(["MasVnrType", "MasVnrArea"], axis=1, inplace=True)
    return data_set

def basement(column_name, data_set, *_):
    options = dict(
        GLQ="Good Living Quarters",
        ALQ="Average Living Quarters",
        BLQ="Below Average Living Quarters",
        Rec="Average Rec Room",
        LwQ="Low Quality",
        Unf="Unfinished",
    )
    for key in options:
        data_set[f"basement {options[key]} sqft"] = 0
        for id in range(len(data_set)):
            if data_set["BsmtFinType1"].iloc[id] == key:
                data_set[f"basement {options[key]} sqft"].iat[id] += data_set["BsmtFinSF1"].iloc[id]
            if data_set["BsmtFinType2"].iloc[id] == key:
                data_set[f"basement {options[key]} sqft"].iat[id] += data_set["BsmtFinSF2"].iloc[id]
    
    data_set.drop(["BsmtFinType1", "BsmtFinSF1", "BsmtFinType2", "BsmtFinSF2"], axis=1, inplace=True)
    return data_set

def basement_unfinished(column_name, data_set, *_):
    unf1 = "basement Unfinished sqft"
    unf2 = "BsmtUnfSF"
    data_set[unf1] = data_set.apply(lambda x: x[unf1] + x[unf2], axis=1)
    
    data_set.drop(["BsmtUnfSF"], axis=1, inplace=True)
    return data_set

def garage_year(column_name, data_set, *_):
    data_set["GarageYrBlt"] = data_set.apply(lambda x: x["YearRemodAdd"] if is_nan(x["GarageYrBlt"]) else x["GarageYrBlt"], axis=1)
    return data_set

def NA_to_zero(column_name, data_set, *_):
    data_set[column_name] = data_set[column_name].apply(lambda x: 0 if is_nan(x) else x)
    return data_set

import inflation
def sale_price(column_name, data_set, *_):
    if column_name in data_set:
        for id in range(len(data_set)):
            data_set[column_name].iat[id] = inflation.adjust(
                price = data_set[column_name].iloc[id],
                from_year = data_set["YrSold"].iloc[id],
                from_month = data_set["MoSold"].iloc[id],
                to_year = 2010,
                to_month = 7,
            )
    data_set.drop(["MoSold", "YrSold"], axis=1, inplace=True)
    return data_set

def post_process(results, data_set, root_data):
    # global original_data
    output = []
    for id, price in enumerate(results):
        id_original = data_set["Id"].iloc[id]
        new_price = inflation.adjust(
            price=price,
            from_year=2010,
            from_month=7,
            to_year=int(root_data["YrSold"][root_data["Id"] == id_original].iloc[0]),
            to_month=int(root_data["MoSold"][root_data["Id"] == id_original].iloc[0]),
        )
        output.append(new_price)
    return results

In [77]:
# specify each column transformation

transformations = {
    "MSSubClass": (drop, None),
    "MSZoning": (MSZoning, None),
    "LotFrontage": (apply, lambda x: 0 if is_nan(x) else x),
    "Street": (re_map, {"Pave": 1, "Grvl": 0}),
    "Alley": (drop, None),
    "LotShape": (re_map, {"Reg": 0, "IR1": 1, "IR2": 2, "IR3": 3}),
    "LandContour": (re_map, {"Lvl": 1, "Bnk": 3, "HLS": 2, "Low": 0}),
    "Utilities": (drop, None),
    "LotConfig": (one_hot, all_unique_values("LotConfig")),
    "LandSlope": (re_map, {"Gtl": 0, "Mod": 1, "Sev": 2}),
    "Neighborhood": (one_hot, all_unique_values("Neighborhood")),
    "Condition1": (drop, None),
    "Condition2": (drop, None),
    "BldgType": (one_hot, all_unique_values("BldgType")),
    "HouseStyle": (one_hot, all_unique_values("HouseStyle")),
    "YearBuilt": (drop, None),
    "RoofStyle": (one_hot, all_unique_values("RoofStyle")),
    "RoofMatl": (one_hot, all_unique_values("RoofMatl")),
    "Exterior1st": (one_hot, all_unique_values("Exterior1st")),
    "Exterior2nd": (drop, None),
    "MasVnrType": (masonry, None),
    "ExterQual": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0}),
    "ExterCond": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0}),
    "Foundation": (one_hot, all_unique_values("Foundation")),
    "BsmtQual": (re_map, {"Ex": 110, "Gd": 95, "TA": 85, "Fa": 75, "Po": 60, "NA": 0, nan: 0}),
    # "BsmtCond": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0, "NA": 2, nan: 2}),
    "BsmtCond": (drop, None),
    "BsmtExposure": (re_map, {"Gd": 3, "Av": 2, "Mn": 1, "No": 0, "NA": 0, nan: 0}),
    "TotalBsmtSF": (basement, None),
    "BsmtUnfSF": (basement_unfinished, None),
    "Heating": (one_hot, all_unique_values("Heating")),
    "HeatingQC": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0}),
    "CentralAir": (re_map, {"Y": 1, "N": 0}),
    "Electrical": (re_map, {"SBrkr": 4, "FuseA": 3, "FuseF": 2, "FuseP": 1, "Mix": 0, nan: 0}),
    "BsmtFullBath": (NA_to_zero, None),
    "BsmtHalfBath": (NA_to_zero, None),
    "KitchenQual": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0, nan: 0}),
    "Functional": (re_map, {"Typ": 7, "Min1": 6, "Min2": 5, "Mod": 4, "Maj1": 3, "Maj2": 2, "Sev": 1, "Sal": 0, nan: 7}),
    "FireplaceQu": (re_map, {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, nan: 3}),
    "GarageType": (one_hot, all_unique_values("GarageType")),
    "GarageYrBlt": (garage_year, None),
    "GarageFinish": (re_map, {"Fin": 2, "RFn": 1, "Unf": 0, "NA": 1, nan: 1}),
    "GarageCars": (NA_to_zero, None),
    "GarageQual": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0, "NA": 2, nan: 2}),
    "GarageCond": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0, "NA": 2, nan: 2}),
    "PavedDrive": (re_map, {"Y": 2, "P": 1, "N": 0}),
    "PoolQC": (re_map, {"Ex": 5, "Gd": 4, "TA": 2, "Fa": 1, nan: 3}),
    "Fence": (one_hot, all_unique_values("Fence")),
    "MiscFeature": (drop, None),
    "SaleType": (one_hot, all_unique_values("SaleType")),
    "SaleCondition": (one_hot, all_unique_values("SaleCondition")),
    "SalePrice": (sale_price, None),
}

In [78]:
# apply column transformations to data
load_data()
for column_name, (transform, option) in transformations.items():
    print("transforming", column_name)
    training___data = transform(column_name, training___data, option)
    validation_data = transform(column_name, validation_data, option)

validation_data

transforming MSSubClass
transforming MSZoning
transforming LotFrontage
transforming Street
transforming Alley
transforming LotShape
transforming LandContour
transforming Utilities
transforming LotConfig
transforming LandSlope
transforming Neighborhood
transforming Condition1
transforming Condition2
transforming BldgType
transforming HouseStyle
transforming YearBuilt
transforming RoofStyle
transforming RoofMatl
transforming Exterior1st
transforming Exterior2nd
transforming MasVnrType
transforming ExterQual
transforming ExterCond
transforming Foundation
transforming BsmtQual
transforming BsmtCond
transforming BsmtExposure
transforming TotalBsmtSF
transforming BsmtUnfSF
transforming Heating
transforming HeatingQC
transforming CentralAir
transforming Electrical
transforming BsmtFullBath
transforming BsmtHalfBath
transforming KitchenQual
transforming Functional
transforming FireplaceQu
transforming GarageType
transforming GarageYrBlt
transforming GarageFinish
transforming GarageCars
transfo

,Id,LotFrontage,LotArea,Street,LotShape,LandContour,LandSlope,OverallQual,OverallCond,YearRemodAdd,ExterQual,ExterCond,BsmtQual,BsmtExposure,TotalBsmtSF,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,zoned_Floating Village Residential,zoned_Residential High Density,zoned_Residential Low Density,zoned_Residential Medium Density,LotConfig_FR3,LotConfig_FR2,LotConfig_CulDSac,LotConfig_Inside,LotConfig_Corner,Neighborhood_BrDale,Neighborhood_Blmngtn,Neighborhood_SawyerW,Neighborhood_IDOTRR,Neighborhood_StoneBr,Neighborhood_MeadowV,Neighborhood_ClearCr,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_SWISU,Neighborhood_Blueste,Neighborhood_Somerst,Neighborhood_Veenker,Neighborhood_OldTown,Neighborhood_Edwards,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_Mitchel,Neighborhood_Crawfor,Neighborhood_BrkSide,Neighborhood_Timber,Neighborhood_Sawyer,Neighborhood_NAmes,Neighborhood_CollgCr,Neighborhood_Gilbert,BldgType_TwnhsE,BldgType_1Fam,BldgType_Duplex,BldgType_Twnhs,BldgType_2fmCon,HouseStyle_1Story,HouseStyle_2Story,HouseStyle_2.5Unf,HouseStyle_SFoyer,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_SLvl,HouseStyle_2.5Fin,RoofStyle_Gambrel,RoofStyle_Shed,RoofStyle_Mansard,RoofStyle_Gable,RoofStyle_Flat,RoofStyle_Hip,RoofMatl_Roll,RoofMatl_WdShngl,RoofMatl_ClyTile,RoofMatl_Metal,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_CompShg,RoofMatl_Membran,Exterior1st_Stucco,Exterior1st_Plywood,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_AsbShng,Exterior1st_Stone,Exterior1st_WdShing,Exterior1st_AsphShn,Exterior1st_Wd Sdng,Exterior1st_HdBoard,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_VinylSd,masonry Brick Common,masonry Brick Face,masonry Cinder Block,masonry Stone,Foundation_CBlock,Foundation_BrkTil,Foundation_Wood,Foundation_Stone,Foundation_PConc,Foundation_Slab,basement Good Living Quarters sqft,basement Average Living Quarters sqft,basement Below Average Living Quarters sqft,basement Average Rec Room sqft,basement Low Quality sqft,basement Unfinished sqft,Heating_OthW,Heating_Grav,Heating_GasW,Heating_Floor,Heating_GasA,Heating_Wall,GarageType_CarPort,GarageType_2Types,GarageType_Basment,GarageType_Detchd,GarageType_BuiltIn,GarageType_nan,GarageType_Attchd,Fence_GdWo,Fence_MnWw,Fence_MnPrv,Fence_GdPrv,Fence_nan,SaleType_New,SaleType_COD,SaleType_ConLw,SaleType_Oth,SaleType_Con,SaleType_CWD,SaleType_ConLD,SaleType_ConLI,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Abnorml,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_Family
368,369,78.0,7800,1,0,1,0,5,6,1954,2,2,85,0,1268,3,1,4,1268,0,0,1268,0,0,1,0,2,1,2,7,7,1,4,1954.0,2,1,244,2,2,2,0,98,0,0,0,0,3,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,200,0,0,0,0,0,0,1,0,0,0,0,0,540,728,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
71,72,69.0,7599,1,0,1,0,4,6,2006,2,2,85,0,845,2,1,4,845,0,0,845,1,0,1,0,2,1,2,4,7,0,3,1987.0,0,2,360,2,2,2,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,565,0,0,0,280,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
708,709,65.0,9018,1,1,1,0,7,5,2007,3,2,95,2,728,4,1,4,728,728,0,1456,0,0,2,1,3,1,3,8,7,1,4,2007.0,2,2,400,2,2,2,100,24,0,0,0,0,3,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,728,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,

In [79]:
training___data.describe()
validation_data.describe()

,Id,LotFrontage,LotArea,Street,LotShape,LandContour,LandSlope,OverallQual,OverallCond,YearRemodAdd,ExterQual,ExterCond,BsmtQual,BsmtExposure,TotalBsmtSF,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,zoned_Floating Village Residential,zoned_Residential High Density,zoned_Residential Low Density,zoned_Residential Medium Density,LotConfig_FR3,LotConfig_FR2,LotConfig_CulDSac,LotConfig_Inside,LotConfig_Corner,Neighborhood_BrDale,Neighborhood_Blmngtn,Neighborhood_SawyerW,Neighborhood_IDOTRR,Neighborhood_StoneBr,Neighborhood_MeadowV,Neighborhood_ClearCr,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_SWISU,Neighborhood_Blueste,Neighborhood_Somerst,Neighborhood_Veenker,Neighborhood_OldTown,Neighborhood_Edwards,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_Mitchel,Neighborhood_Crawfor,Neighborhood_BrkSide,Neighborhood_Timber,Neighborhood_Sawyer,Neighborhood_NAmes,Neighborhood_CollgCr,Neighborhood_Gilbert,BldgType_TwnhsE,BldgType_1Fam,BldgType_Duplex,BldgType_Twnhs,BldgType_2fmCon,HouseStyle_1Story,HouseStyle_2Story,HouseStyle_2.5Unf,HouseStyle_SFoyer,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_SLvl,HouseStyle_2.5Fin,RoofStyle_Gambrel,RoofStyle_Shed,RoofStyle_Mansard,RoofStyle_Gable,RoofStyle_Flat,RoofStyle_Hip,RoofMatl_Roll,RoofMatl_WdShngl,RoofMatl_ClyTile,RoofMatl_Metal,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_CompShg,RoofMatl_Membran,Exterior1st_Stucco,Exterior1st_Plywood,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_AsbShng,Exterior1st_Stone,Exterior1st_WdShing,Exterior1st_AsphShn,Exterior1st_Wd Sdng,Exterior1st_HdBoard,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_VinylSd,masonry Brick Common,masonry Brick Face,masonry Cinder Block,masonry Stone,Foundation_CBlock,Foundation_BrkTil,Foundation_Wood,Foundation_Stone,Foundation_PConc,Foundation_Slab,basement Good Living Quarters sqft,basement Average Living Quarters sqft,basement Below Average Living Quarters sqft,basement Average Rec Room sqft,basement Low Quality sqft,basement Unfinished sqft,Heating_OthW,Heating_Grav,Heating_GasW,Heating_Floor,Heating_GasA,Heating_Wall,GarageType_CarPort,GarageType_2Types,GarageType_Basment,GarageType_Detchd,GarageType_BuiltIn,GarageType_nan,GarageType_Attchd,Fence_GdWo,Fence_MnWw,Fence_MnPrv,Fence_GdPrv,Fence_nan,SaleType_New,SaleType_COD,SaleType_ConLw,SaleType_Oth,SaleType_Con,SaleType_CWD,SaleType_ConLD,SaleType_ConLI,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Abnorml,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_Family
count,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.000000,365.0,365.000000,365.000000,365.000000,365.000000,365.0,365.00000

# Fit, cross-validate, & compare

In [80]:
from xgboost import XGBRegressor
from sigfig import round
pretty = lambda x: round(str(x), decimals=2, spacer=",", spacing=3)
from sklearn.metrics import mean_absolute_error
from capture_output import Capturing
from random import shuffle

load_data()
for column_name, (transform, option) in transformations.items():
    full_data = transform(column_name, full_data, option)

def generate_cross_validation_sets(full_data, sets=5):
    indicies = [i for i in range(len(full_data))]
    shuffle(indicies)
    set_size = len(full_data) // sets
    for n in range(sets):
        yield indicies[n*set_size:(n+1)*set_size]

errors = []
iterations = []
for indicies_subset in generate_cross_validation_sets(full_data, sets=5):
    training___data = full_data.drop(indicies_subset)
    training___price = full_price.drop(indicies_subset)
    validation_data = full_data.iloc[indicies_subset]
    validation_price = full_price.iloc[indicies_subset]

    model = None
    model = XGBRegressor(n_estimators=1_000, learning_rate=0.05, early_stopping_rounds=5)
    with Capturing() as output:
        model.fit(training___data.drop("Id", axis=1), training___price, eval_set=[(validation_data.drop("Id", axis=1), validation_price)])

    predictions = model.predict(validation_data.drop("Id", axis=1))
    predictions = post_process(predictions, validation_data, original_data)

    error = float(mean_absolute_error(validation_price, predictions))
    print(f"Error: {pretty(error)}", f"Iterations: {len(output)}")
    errors.append(error)
    iterations.append(len(output))

print(f"Mean Absolute Error: {pretty(mean(errors))}")
print(f"Optimal # Iterations(?): {mean(iterations)}")

Error: 15,776.64 Iterations: 145
Error: 17,224.86 Iterations: 173
Error: 14,581.78 Iterations: 199
Error: 18,784.37 Iterations: 123
Error: 15,685.67 Iterations: 167
Mean Absolute Error: 16,410.66
Optimal # Iterations(?): 161.4


## Saved results (with Random Forest model)

- Normie: Mean Absolute Error: 17787.279715068493: 16742.85882191781...19406.53876712329
- No Alley: Mean Absolute Error of 17652.627992694062: 16217.109150684932...19793.63994520548
- Basic zoning: Mean Absolute Error of 18059.097720547943: 16746.11709589041...19786.975287671234

# Generate Output

In [83]:
from xgboost import XGBRegressor

load_data()
for column_name, (transform, option) in transformations.items():
    full_data = transform(column_name, full_data, option)
    test_data = transform(column_name, test_data, option)

model = None
model = XGBRegressor(n_estimators=1_000, learning_rate=0.05)
model.fit(full_data.drop("Id", axis=1), full_price)

predictions = model.predict(test_data.drop("Id", axis=1))
predictions = post_process(predictions, test_data, original_test_data)

submission = pd.DataFrame({'Id': test_data.index + 1461, 'SalePrice': predictions})
submission.to_csv('submission.csv', index=False)
submission

,Id,SalePrice
0,1461,122960.750000
1,1462,167588.031250
2,1463,181527.718750
3,1464,190160.578125
4,1465,203490.656250
...,...,...
1454,2915,78206.265625
1455,2916,83949.570312
1456,2917,157892.312500
1457,2918,114367.906250


In [82]:
original_test_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,NaN,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960.0,Unf,2.0,576.0,TA,TA,Y,474,0,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,5,1992,1992,Gable,CompShg,Hd